In [14]:
# environment set up
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import ast

# working folder
# directory = "/Users/Srikar/Desktop/Velexi/spectra-ml/data"
data_dir = os.environ['DATA_DIR']
os.chdir(data_dir)
stddata_path = os.path.join(data_dir,"Srikar-Standardized")

# print(os.getcwd())

In [15]:
data = pd.read_csv("/Users/Srikar/Desktop/Velexi/spectra-ml/lab-notebook/smunukutla/data.csv", sep=",")
record_nums = data.iloc[0, :].tolist()
spectrum_names = data.iloc[1, :].tolist()
y = data.iloc[2, :].astype(int).tolist()
y = np.reshape(y, (len(y), 1))
num_samples = len(y)

In [16]:
spectrum_len = 500
spectra = np.zeros((num_samples,spectrum_len))
wavelengths = np.zeros((1,spectrum_len))

# y = np.zeros((num_samples, 1))

In [17]:
for i in range(len(record_nums)):
    data = pd.read_csv(os.path.join(stddata_path,"{}.csv".format(record_nums[i])))
    if i == 0:
        wavelengths[i,:] = data.iloc[:, 0].to_numpy()
    spectra[i,:] = data.iloc[:, 1].to_numpy()

In [18]:
# spectrum_len = 480
# spectra = np.zeros((num_samples,spectrum_len))

# spectrum_categories = np.zeros(num_samples)
# first_record_of_mixtures_chapter = 11602
# is_a_mineral = 1                                   # these numbers match the chapter numbers given by usgs
# is_a_mixture = 2
# spectrum_names = ["" for x in range(num_samples)]

# y = np.zeros((num_samples, 1))

# os.chdir(dataset)

# i = 0

# for filename in os.listdir(dataset):
#     file_object  = open(filename, 'r').readlines()
# #     strip off header, add to matrix 'spectra'
#     spectra[i,:] = file_object[1:]

# #     label spectrum class, based on header
# #     actinolite: 0, alunite: 1, chlorite: 2
#     material_name = file_object[0]
    
#     spectrum_names[i] = material_name
    
#     start = 'Record='
#     end = ':'
#     record_number = int((material_name.split(start))[1].split(end)[0])
#     # print(record_number)
#     if record_number < first_record_of_mixtures_chapter:
#         spectrum_categories[i] = is_a_mineral
#     else:
#         spectrum_categories[i] = is_a_mixture

# #     print(material_name)

#     if material_name.find('Actinolite',) != -1: # if material name contains actinolite
#         y[i,0] = 0
#     elif material_name.find('Alun',)!= -1:
#         y[i,0] = 1
#     else: # chlorite
#         y[i,0] = 2

# #     turn missing points into 0
#     for j in range(spectrum_len):
#         if spectra[i,j] < 0:
#             spectra[i,j] = 0
#     i+=1

In [19]:
# --- plot the classes

# plot each class in a separate plot
# plot spectra names in legend
# plot minerals and mixtures w diff line widths

mineral_names = ["Actinolite", "Alunite", "Chlorite"]

# variables
num0 = 0 #number of samples of class 0
num1 = 0
num2 = 0

mineral_linewidth = 1         # linewidth = 1 is default
mixture_linewidth = 3         

# count the number of each class to make spectra0, spectra1, spectra2 databases
for i in range(num_samples):
    if y[i,0]== 0:
        num0 += 1
    elif y[i,0]== 1:
        num1 += 1
    elif y[i,0]== 2:
        num2 += 1

# make class-specific databases spectra0, ...1, ...2
spectra0 = np.zeros((num0,spectrum_len)) 
spectra1 = np.zeros((num1,spectrum_len)) 
spectra2 = np.zeros((num2,spectrum_len)) 

labels0 = ["" for x in range(num0)]
labels1 = ["" for x in range(num1)]
labels2 = ["" for x in range(num2)]

linewidth0 = np.zeros(num0)
linewidth1 = np.zeros(num1)
linewidth2 = np.zeros(num2)


# make counters for each database to place spectra
i0 = 0
i1 = 0
i2 = 0

# set linewidth for the spectrum 
# populate class-specific databases spectra0, ...1, ...2
for i in range(num_samples):
    
    # set linewidth
    #testcode
    #print(spectrum_categories)
    #print(spectrum_categories[i])
    
#     if spectrum_categories[i] == is_a_mineral:
#         linewidth = mineral_linewidth
        
#         #testcode
#         #print('min')
#     else: 
#         linewidth = mixture_linewidth
    linewidth = 2
        
        #testcode
        #print('mix')
    
    # populate matrices for making each class plot
    if y[i,0]== 0:
        spectra0[i0,:] = spectra[i,:]
        labels0[i0] = spectrum_names[i]
        linewidth0[i0] = linewidth
        i0 +=1
    elif y[i,0]== 1:
        spectra1[i1,:] = spectra[i,:]
        labels1[i1] = spectrum_names[i]
        linewidth1[i1] = linewidth
        i1 +=1
    else:
        spectra2[i2,:] = spectra[i,:]
        labels2[i2] = spectrum_names[i]
        linewidth2[i2] = linewidth
        i2 +=1

# plot each class-specific database separately
# for i in range(i0):
#     fig = plt.figure()
#     plt.plot(range(1, spectrum_len+1), spectra0[i,:], label = labels0[i], linewidth = linewidth0[i]) # remove linewidth for all mixtures/minerals to be standard
#     plt.plot(wavelengths[0,:], spectra0[i,:]) # remove linewidth for all mixtures/minerals to be standard
#     plt.xticks([])
#     plt.yticks([])
#     plt.show()
#     path = "/Users/Srikar/Desktop/Velexi/spectra-ml/lab-notebook/smunukutla/plots/" + mineral_names[0] + str(i) + ".png"
#     fig.savefig(path, format = "PNG")
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

# for i in range(i1):
#     plt.plot(range(1, spectrum_len+1), spectra1[i,:], label = labels1[i], linewidth = linewidth1[i])
#     plt.plot(wavelengths[0,:], spectra1[i,:])
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

# for i in range(i2):
#     plt.plot(range(1, spectrum_len+1), spectra2[i,:], label = labels2[i], linewidth = linewidth2[i])
#     plt.plot(wavelengths[0,:], spectra2[i,:])
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

In [40]:
os.chdir("/Users/Srikar/Desktop/Velexi/spectra-ml/lab-notebook/smunukutla")
fi = open("indices.txt", "r")

for i in range(10):
    train_set_indices = ast.literal_eval(fi.readline())
    test_set_indices = ast.literal_eval(fi.readline())
    dev_set_indices = ast.literal_eval(fi.readline())
    
    for j in train_set_indices:
        j = int(j)
    for k in test_set_indices:
        k = int(k)
    for m in dev_set_indices:
        m = int(m)
    
    train_set = spectra[train_set_indices, :]
    train_labels = y[train_set_indices, :]
    dev_set = spectra[dev_set_indices, :]
    dev_labels = y[dev_set_indices, :]
    test_set = spectra[test_set_indices, :]
    test_labels = y[test_set_indices, :]

    train_labels = train_labels.flatten()
    dev_labels = dev_labels.flatten()
    test_labels = test_labels.flatten()

    train_set = np.reshape(train_set, (train_set.shape[0], spectrum_len, 1))
    dev_set = np.reshape(dev_set, (dev_set.shape[0], spectrum_len, 1))
    test_set = np.reshape(test_set, (test_set.shape[0], spectrum_len, 1))

    train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
    dev_labels = np.reshape(dev_labels, (dev_labels.shape[0], 1))
    test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

    train_labels = to_categorical(train_labels)
    dev_labels = to_categorical(dev_labels)
    test_labels = to_categorical(test_labels)
    
    model = Sequential()
    # model.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
    model.add(Conv1D(64, 25, activation='relu', input_shape=(train_set.shape[1], 1)))
    model.add(Conv1D(64, 25, activation='relu'))
    model.add(MaxPooling1D(4)) # 108 by 64 so far
    model.add(Conv1D(100, 25, activation='relu'))
    model.add(Conv1D(100, 25, activation='relu'))
    model.add(MaxPooling1D(4))
    # model.add(Dropout(0.5))
    # model.add(GlobalAveragePooling1D())
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    BATCH_SIZE = 12
    EPOCHS = 50

    model.fit(train_set, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(dev_set, dev_labels))
    
    model.evaluate(test_set, test_labels)

Train on 24 samples, validate on 8 samples
Epoch 1/50
24/24 [==============================] - 1s 36ms/sample - loss: 1.0897 - acc: 0.3750 - val_loss: 0.9338 - val_acc: 0.7500
Epoch 2/50
24/24 [==============================] - 0s 9ms/sample - loss: 1.2583 - acc: 0.6667 - val_loss: 0.8354 - val_acc: 0.7500
Epoch 3/50
24/24 [==============================] - 0s 9ms/sample - loss: 1.0166 - acc: 0.6250 - val_loss: 1.0222 - val_acc: 0.5000
Epoch 4/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.9504 - acc: 0.6667 - val_loss: 0.9867 - val_acc: 0.7500
Epoch 5/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.8789 - acc: 0.7500 - val_loss: 0.8221 - val_acc: 0.8750
Epoch 6/50
24/24 [==============================] - 0s 10ms/sample - loss: 0.7237 - acc: 0.7500 - val_loss: 0.5593 - val_acc: 0.8750
Epoch 7/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.6378 - acc: 0.7083 - val_loss: 0.5665 - val_acc: 0.7500
Epoch 8/50
24/24 [=============

24/24 [==============================] - 0s 8ms/sample - loss: 0.4178 - acc: 0.7500 - val_loss: 0.5343 - val_acc: 0.7500
Epoch 12/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.4023 - acc: 0.7500 - val_loss: 0.6027 - val_acc: 0.7500
Epoch 13/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.3294 - acc: 0.9167 - val_loss: 0.5889 - val_acc: 0.7500
Epoch 14/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.3342 - acc: 0.8750 - val_loss: 0.5322 - val_acc: 0.7500
Epoch 15/50
24/24 [==============================] - 0s 7ms/sample - loss: 0.2555 - acc: 0.9167 - val_loss: 0.6797 - val_acc: 0.5000
Epoch 16/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.2199 - acc: 0.9583 - val_loss: 0.4158 - val_acc: 0.8750
Epoch 17/50
24/24 [==============================] - 0s 7ms/sample - loss: 0.1748 - acc: 0.9583 - val_loss: 0.5540 - val_acc: 0.7500
Epoch 18/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.1268

Epoch 22/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.5215 - acc: 0.8333 - val_loss: 0.7884 - val_acc: 0.3750
Epoch 23/50
24/24 [==============================] - 0s 12ms/sample - loss: 0.6189 - acc: 0.6667 - val_loss: 0.7376 - val_acc: 0.5000
Epoch 24/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.4262 - acc: 0.8333 - val_loss: 0.4674 - val_acc: 0.8750
Epoch 25/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.5320 - acc: 0.6667 - val_loss: 0.4822 - val_acc: 0.8750
Epoch 26/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.4663 - acc: 0.7917 - val_loss: 0.4709 - val_acc: 0.8750
Epoch 27/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.3276 - acc: 1.0000 - val_loss: 0.4228 - val_acc: 0.8750
Epoch 28/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.2689 - acc: 0.9167 - val_loss: 0.3486 - val_acc: 0.8750
Epoch 29/50
24/24 [==============================] - 0s 8ms/sample -

24/24 [==============================] - 0s 8ms/sample - loss: 0.1268 - acc: 0.9167 - val_loss: 0.3854 - val_acc: 0.8750
Epoch 33/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.0784 - acc: 0.9583 - val_loss: 0.2787 - val_acc: 0.8750
Epoch 34/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.0795 - acc: 0.9583 - val_loss: 0.2991 - val_acc: 0.8750
Epoch 35/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.1100 - acc: 0.9583 - val_loss: 0.1906 - val_acc: 0.8750
Epoch 36/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.0295 - acc: 1.0000 - val_loss: 0.1723 - val_acc: 0.8750
Epoch 37/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.0420 - acc: 1.0000 - val_loss: 0.2086 - val_acc: 0.8750
Epoch 38/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.0669 - acc: 1.0000 - val_loss: 0.1667 - val_acc: 0.8750
Epoch 39/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.0356

Epoch 43/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.0131 - acc: 1.0000 - val_loss: 1.4176 - val_acc: 0.8750
Epoch 44/50
24/24 [==============================] - 0s 11ms/sample - loss: 0.0101 - acc: 1.0000 - val_loss: 1.1401 - val_acc: 0.8750
Epoch 45/50
24/24 [==============================] - 0s 7ms/sample - loss: 0.0094 - acc: 1.0000 - val_loss: 0.9793 - val_acc: 0.8750
Epoch 46/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.0163 - acc: 1.0000 - val_loss: 0.8734 - val_acc: 0.8750
Epoch 47/50
24/24 [==============================] - 0s 7ms/sample - loss: 0.0140 - acc: 1.0000 - val_loss: 0.9931 - val_acc: 0.8750
Epoch 48/50
24/24 [==============================] - 0s 11ms/sample - loss: 0.0138 - acc: 1.0000 - val_loss: 1.4596 - val_acc: 0.7500
Epoch 49/50
24/24 [==============================] - 0s 7ms/sample - loss: 0.0635 - acc: 0.9583 - val_loss: 1.6264 - val_acc: 0.8750
Epoch 50/50
8/8 [==============================] - 0s 2ms/sample - 

Epoch 3/50
24/24 [==============================] - 0s 9ms/sample - loss: 1.0693 - acc: 0.5000 - val_loss: 0.9420 - val_acc: 0.5000
Epoch 4/50
24/24 [==============================] - 0s 14ms/sample - loss: 1.0340 - acc: 0.4583 - val_loss: 1.0354 - val_acc: 0.7500
Epoch 5/50
24/24 [==============================] - 0s 9ms/sample - loss: 1.0535 - acc: 0.6250 - val_loss: 1.0438 - val_acc: 0.7500
Epoch 6/50
24/24 [==============================] - 0s 8ms/sample - loss: 1.0509 - acc: 0.7500 - val_loss: 1.0290 - val_acc: 0.7500
Epoch 7/50
24/24 [==============================] - 0s 12ms/sample - loss: 1.0341 - acc: 0.7500 - val_loss: 0.9825 - val_acc: 0.7500
Epoch 8/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.9777 - acc: 0.7083 - val_loss: 0.8939 - val_acc: 0.7500
Epoch 9/50
24/24 [==============================] - 1s 21ms/sample - loss: 0.8840 - acc: 0.7083 - val_loss: 0.7552 - val_acc: 0.7500
Epoch 10/50
24/24 [==============================] - 0s 10ms/sample - los

24/24 [==============================] - 0s 9ms/sample - loss: 0.4069 - acc: 0.7500 - val_loss: 0.3776 - val_acc: 1.0000
Epoch 14/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.3842 - acc: 0.7917 - val_loss: 0.4574 - val_acc: 0.6250
Epoch 15/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.3655 - acc: 0.8750 - val_loss: 0.5056 - val_acc: 0.6250
Epoch 16/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.3991 - acc: 0.8333 - val_loss: 0.4681 - val_acc: 0.6250
Epoch 17/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.3289 - acc: 0.9167 - val_loss: 0.3563 - val_acc: 0.8750
Epoch 18/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.3312 - acc: 0.8333 - val_loss: 0.2804 - val_acc: 1.0000
Epoch 19/50
24/24 [==============================] - 0s 12ms/sample - loss: 0.3405 - acc: 0.7917 - val_loss: 0.3456 - val_acc: 0.7500
Epoch 20/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.339

24/24 [==============================] - 0s 8ms/sample - loss: 0.1790 - acc: 0.9167 - val_loss: 0.1568 - val_acc: 0.8750
Epoch 24/50
24/24 [==============================] - 0s 11ms/sample - loss: 0.3158 - acc: 0.8333 - val_loss: 0.0649 - val_acc: 1.0000
Epoch 25/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.1454 - acc: 0.9167 - val_loss: 0.3339 - val_acc: 0.8750
Epoch 26/50
24/24 [==============================] - 0s 9ms/sample - loss: 0.4289 - acc: 0.8750 - val_loss: 0.1385 - val_acc: 0.8750
Epoch 27/50
24/24 [==============================] - 0s 12ms/sample - loss: 0.1214 - acc: 0.9583 - val_loss: 0.0986 - val_acc: 1.0000
Epoch 28/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.2421 - acc: 0.8750 - val_loss: 0.0901 - val_acc: 1.0000
Epoch 29/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.1600 - acc: 0.9583 - val_loss: 0.0746 - val_acc: 1.0000
Epoch 30/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.17

Epoch 34/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.3919 - acc: 0.9167 - val_loss: 0.3469 - val_acc: 0.8750
Epoch 35/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.3416 - acc: 0.9583 - val_loss: 0.3368 - val_acc: 0.8750
Epoch 36/50
24/24 [==============================] - 0s 7ms/sample - loss: 0.3077 - acc: 0.8750 - val_loss: 0.3027 - val_acc: 0.8750
Epoch 37/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.2356 - acc: 0.9583 - val_loss: 0.3440 - val_acc: 0.7500
Epoch 38/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.2223 - acc: 0.9583 - val_loss: 0.3893 - val_acc: 0.7500
Epoch 39/50
24/24 [==============================] - 0s 12ms/sample - loss: 0.1605 - acc: 1.0000 - val_loss: 0.5204 - val_acc: 0.7500
Epoch 40/50
24/24 [==============================] - 0s 8ms/sample - loss: 0.1202 - acc: 1.0000 - val_loss: 0.3364 - val_acc: 0.8750
Epoch 41/50
24/24 [==============================] - 0s 8ms/sample -

In [38]:
# train_set = np.reshape(train_set, (train_set.shape[0], spectrum_len, 1))
# dev_set = np.reshape(dev_set, (dev_set.shape[0], spectrum_len, 1))
# test_set = np.reshape(test_set, (test_set.shape[0], spectrum_len, 1))

# train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
# dev_labels = np.reshape(dev_labels, (dev_labels.shape[0], 1))
# test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

# train_labels = to_categorical(train_labels)
# dev_labels = to_categorical(dev_labels)
# test_labels = to_categorical(test_labels)

In [37]:
# # random.seed()
# model = Sequential()
# # model.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
# model.add(Conv1D(64, 25, activation='relu', input_shape=(train_set.shape[1], 1)))
# model.add(Conv1D(64, 25, activation='relu'))
# model.add(MaxPooling1D(4)) # 108 by 64 so far
# model.add(Conv1D(100, 25, activation='relu'))
# model.add(Conv1D(100, 25, activation='relu'))
# model.add(MaxPooling1D(4))
# # model.add(Dropout(0.5))
# # model.add(GlobalAveragePooling1D())
# model.add(Flatten())
# model.add(Dense(3, activation='softmax'))
# print(model.summary())

In [36]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# BATCH_SIZE = 12
# EPOCHS = 50

# print(train_labels.shape)
# model.fit(train_set, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(dev_set, dev_labels)) 

In [33]:
# y_pred = model.predict(test_set)
# y_pred

In [34]:
# test_labels

In [35]:
# model.evaluate(test_set, test_labels)